In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
tf.random.set_seed(42)
np.random.seed(42)

## Parameters

In [2]:
# Bits per Symbol
k = 4

# Number of symbols

L = 50
# Channel Use
n = 3

# Effective Throughput
#  (bits per symbol)*( number of symbols) / channel use
R = k / n

# Eb/N0 used for training
train_Eb_dB = 12

# Noise Standard Deviation
noise_sigma = np.sqrt(1 / (2 * R * 10 ** (train_Eb_dB / 10)))

# Number of messages used for training, each size = k*L
batch_size = 64
nb_train_word = batch_size*k*L

## Generating the binary sequence and converting to one-hot vectors

In [3]:
# Generate Binary Sequence
train_data = tf.random.uniform(shape=(nb_train_word, k*L), minval=0, maxval=2, dtype=tf.dtypes.int32)
train_data

<tf.Tensor: shape=(12800, 200), dtype=int32, numpy=
array([[1, 1, 1, ..., 0, 1, 0],
       [0, 0, 0, ..., 1, 1, 1],
       [0, 0, 1, ..., 1, 1, 1],
       ...,
       [0, 0, 1, ..., 0, 0, 1],
       [1, 0, 0, ..., 0, 1, 0],
       [1, 0, 1, ..., 1, 0, 0]], dtype=int32)>

In [5]:
# Reshaping into N x L x k (N: number of data messages, L: block length, k: bits/symbol)
train_data = tf.reshape(shape=(nb_train_word, L, k), tensor=train_data)
train_data.shape

TensorShape([12800, 50, 4])

In [22]:
def BitsToInt(bits, k):
    '''Function to transform a binary sequence into integers.'''
    a = 2**np.arange(k)[::-1]
    a = np.reshape(a, newshape=(k,1))
    return bits @ a

tmp = BitsToInt(train_data.numpy(),k)
print('Integers samples Shape:', tmp.shape)
one_hot_train = tf.keras.utils.to_categorical(y=tmp, num_classes=2 ** k)
print('One-Hot TRAINING Shape:', one_hot_train.shape)

Integers samples Shape: (12800, 50, 1)
One-Hot TRAINING Shape: (12800, 50, 16)


## Saving the labels

In [21]:
one_hot_labels = tf.identity(one_hot_train)
print('One-Hot LABELS Shape:', one_hot_labels.shape)

One-Hot LABELS Shape: (12800, 50, 16)


## Convolutional Neural Networks

In [25]:
early_stopping_patience = 100

epochs = 250

optimizer = tf.keras.optimizers.Adam(lr=0.001)

early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                               patience=early_stopping_patience)


# Learning Rate Control
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='loss', factor=0.1,
                              patience=5, min_lr=0.0001)

# Save the best results based on Training Set
modelcheckpoint = tf.keras.callbacks.ModelCheckpoint(filepath='./' + 'model_' + str(k) + '_' + str(L) + '_' + str(n) + '_' + str(train_Eb_dB) + 'dB' + ' ' + 'Rayleigh ' + '.h5',
                                  monitor='loss',
                                  verbose=1,
                                  save_best_only=True,
                                  save_weights_only=True,
                                  mode='auto', save_freq=1)
